In [45]:
# Filter rows where 'rel_diff_to_min_price_JourneyID' is greater than 20
values_larger_than_20 = df_train[df_train['rel_diff_to_min_price_FlightID'] > 50]

# Display the filtered DataFrame
print(values_larger_than_20)


     airline_code departure_airport_code destination_airport_code  \
498         multi                    PHX                      CGH   
500         multi                    PHX                      CGH   
1006           AC                    LAX                      LIS   
1007           AC                    LAX                      LIS   
1008           AC                    LAX                      LIS   
1016           UA                    LAX                      LIS   
1017           UA                    LAX                      LIS   
1025           UA                    LAX                      LIS   
1026           UA                    LAX                      LIS   
2088           QR                    MAN                      CPT   
2097           QR                    MAN                      CPT   
3230        multi                    MAN                      CUN   
3231        multi                    MAN                      CUN   
3232        multi                 

In [36]:
df_train.rel_diff_to_min_price_FlightID.describe()

count    5806.000000
mean        3.090059
std         8.683293
min         0.000000
25%         0.626945
50%         1.648590
75%         3.930153
max       257.689565
Name: rel_diff_to_min_price_FlightID, dtype: float64

In [40]:
df_train.rel_diff_to_min_price_JourneyID.describe()

count    5806.000000
mean       50.175693
std       115.048079
min         0.000000
25%         4.122248
50%        17.562475
75%        53.030424
max      1187.102731
Name: rel_diff_to_min_price_JourneyID, dtype: float64

In [29]:
from scipy import stats
trimmed_mean = stats.trim_mean(df_train['rel_diff_to_min_price_JourneyID'], proportiontocut=0.3)
trimmed_mean

19.473083666012283

In [23]:
raw_mean = df_train['rel_diff_to_min_price_JourneyID'].mean()
std_dev = df_train['rel_diff_to_min_price_JourneyID'].std()


In [26]:
raw_mean

50.1756933492957

In [24]:
smoothing_factor = std_dev  # This is a simple choice; adjust based on your data's characteristics


In [30]:
global_median_savings = df_train['rel_diff_to_min_price_JourneyID'].median()


In [31]:
global_median_savings

17.56247473048319

In [15]:
category_medians = df_train.groupby(['Journey_route', 'Detected_Country'])['rel_diff_to_min_price_JourneyID'].median()
category_counts = df_train.groupby(['Journey_route', 'Detected_Country']).size()


In [16]:
smoothing_factor = 10  # This is a hyperparameter you can tune

# Calculate the smoothed average for each category
smoothed_medians = ((category_counts * category_medians) + (smoothing_factor * global_median_savings)) / (category_counts + smoothing_factor)

# Convert the Series to a DataFrame for merging
smoothed_medians_df = smoothed_medians.reset_index(name='smoothed_median_savings_for_Journey_ID_in_Detected_Country')


In [17]:
df_train = pd.merge(df_train, smoothed_medians_df, on=['Journey_route', 'Detected_Country'], how='left')


In [18]:
df_train.smoothed_median_savings_for_Journey_ID_in_Detected_Country.describe()

count    5806.000000
mean       23.416404
std        17.244218
min         7.571674
25%        13.950521
50%        18.820470
75%        26.517228
max       300.450249
Name: smoothed_median_savings_for_Journey_ID_in_Detected_Country, dtype: float64

In [22]:
df_train.median_savings_for_Journey_ID_in_Detected_Country.describe()

count    5806.000000
mean       27.186593
std        35.248063
min         0.000000
25%        10.734351
50%        19.190072
75%        29.329962
max      1172.493976
Name: median_savings_for_Journey_ID_in_Detected_Country, dtype: float64

# Import training data

In [1]:
import pandas as pd
df_1 = pd.read_csv('../data/4.processed_data/Query4_results_test_processed.csv')
df_2 = pd.read_csv('../data/4.processed_data/Query1503_results_conv_processed.csv')
df_3 = pd.read_csv('../data/4.processed_data/Query1603_results_test_processed.csv')
df_train = pd.concat([df_1,df_2,df_3])
df_test = pd.read_csv('../data/4.processed_data/Query1803_results_processed.csv')

In [21]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_route_train = df_train.groupby(['Journey_route', 'Detected_Country','Detected_Currency'])['rel_diff_to_min_price_JourneyID'].median().reset_index(name='median_savings_for_Journey_ID_in_Detected_Country')
# Merge the average_savings DataFrame back into df_reduced
df_train= df_train.merge(median_savings_route_train, on=['Journey_route', 'Detected_Country','Detected_Currency'], how='left')

In [34]:
df_train.rel_diff_to_min_price_JourneyID.median()

17.56247473048319

In [3]:
median_savings_route_train.describe()

,median_savings_for_Journey_ID_in_Detected_Country
count,383.000000
mean,31.231280
std,70.564027
min,0.000000
25%,8.707555
50%,15.823207
75%,39.232218
max,1172.493976


In [4]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_route_test = df_test.groupby(['Journey_route', 'Detected_Country'])['rel_diff_to_min_price_JourneyID'].median().reset_index(name='median_savings_for_Journey_ID_in_Detected_Country')
# Merge the average_savings DataFrame back into df_reduced
df_test = df_test.merge(median_savings_route_test, on=['Journey_route', 'Detected_Country'], how='left')

In [5]:
df_test.columns

Index(['airline_code', 'departure_airport_code', 'destination_airport_code',
       'ticket_price', 'departure_date', 'arrival_date', 'First_flight',
       'first_flight_code', 'last_flight_code', 'Detected_Language',
       'Detected_Country', 'Detected_Currency', 'Flight_ID', 'Price_in_USD',
       'commute_time', 'query_date', 'days_until_departure',
       'FlightID_in_Countries_Count', 'departure_date_day', 'arrival_date_day',
       'Journey_route', 'Journey_ID', 'max_price_FlightID',
       'min_price_FlightID', 'max_price_diff_FlightID',
       'max_rel_price_diff_FlightID', 'abs_diff_to_min_price_FlightID',
       'rel_diff_to_min_price_FlightID', 'rel_price_score_FlightID',
       'max_price_JourneyID', 'min_price_JourneyID', 'max_abs_diff_JourneyID',
       'max_rel_diff_Journey', 'abs_diff_to_min_price_JourneyID',
       'rel_diff_to_min_price_JourneyID', 'rel_price_score_JourneyID',
       'max_journey_same_country', 'min_journey_same_country',
       'max_abs_diff_perIDG

In [6]:

import xgboost as xgb

X_full = pd.get_dummies(df_train[['Journey_route','days_until_departure', 'Detected_Country', 'Detected_Currency']])
y_full = df_train['median_savings_for_Journey_route_in_Detected_Country']
feature_names = X_full.columns.tolist()



# Specify your model parameters here
params_grid = {'colsample_bytree': 0.9, 'eta': 0.299, 'max_depth': 7, 'subsample': 0.8}
params_optuna = {'max_depth': 7, 'eta': 0.319923163390001, 'subsample': 0.8, 'colsample_bytree': 0.9}
params_optuna200 = {'max_depth': 8, 'eta': 0.34991201767803204, 'subsample': 0.9, 'colsample_bytree': 0.9}


params_self = {'max_depth': 3, 'eta': 0.1, 'subsample': 0.9, 'colsample_bytree': 0.9}



median_model = xgb.XGBRegressor(**params_self)


# Fit the model on your training data
median_model.fit(X_full, y_full)

KeyError: 'median_savings_for_Journey_route_in_Detected_Country'

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# Assuming new_test_df is your new test data

#df_test18 = pd.read_csv("../data/4.processed_data/Query1603_results_test_processed.csv")
X_new_test = pd.get_dummies(df_test[['Journey_route','days_until_departure', 'Detected_Country', 'Detected_Currency','airline_code']])
y_test = df_test['median_savings_for_Journey_route_in_Detected_Country']


# Align the new test set features with the training features
# Add missing columns in the new test set
missing_cols = set(feature_names) - set(X_new_test.columns)
for c in missing_cols:
    X_new_test[c] = 0

# Remove any columns in the new test set that are not present in the training set
X_new_test = X_new_test[feature_names]

# Now, you can predict with your model
y_pred = median_model.predict(X_new_test)



# Calculate and print the Mean Squared Error
mea = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mea}")


r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 9.299601193832142
R-squared: 0.6831993250218229


In [ ]:
y_pred

array([15.101352 , 15.101352 , 15.101352 , 15.101352 , 15.101352 ,
       15.101352 , 15.101352 , 16.951729 , 15.101352 , 15.101352 ,
       15.101352 , 15.101352 , 15.101352 , 15.101352 , 15.101352 ,
       16.951729 , 15.101352 , 15.101352 , 15.101352 , 15.101352 ,
       15.101352 , 15.101352 , 15.101352 , 16.951729 , 15.101352 ,
       15.101352 , 15.101352 , 15.101352 , 15.101352 , 15.101352 ,
       15.101352 , 16.951729 , 15.101352 , 15.101352 , 15.101352 ,
       15.101352 , 15.101352 , 15.101352 , 15.101352 , 16.951729 ,
       15.101352 , 15.101352 , 15.101352 , 15.101352 , 15.101352 ,
       15.101352 , 15.101352 , 16.951729 , 15.101352 , 15.101352 ,
       15.101352 , 15.101352 , 15.101352 , 15.101352 , 15.101352 ,
       16.951729 , 19.62722  , 19.62722  , 19.62722  , 19.62722  ,
       19.62722  , 19.62722  , 19.62722  , 19.62722  , 19.62722  ,
       19.62722  , 19.62722  , 19.62722  , 19.62722  , 19.62722  ,
       20.399437 , 19.62722  , 19.62722  , 19.62722  , 19.6272

In [ ]:
y_test

0      7.109625
1      6.568892
2      6.516954
3      6.880284
4      6.848363
         ...   
699    2.744462
700    2.876564
701    3.303558
702    2.662714
703    2.819764
Name: median_savings_for_Journey_route_in_Detected_Country, Length: 704, dtype: float64

In [ ]:
df_train.median_savings_for_Journey_route_in_Detected_Country.describe()

count    5806.000000
mean       27.186593
std        35.248063
min         0.000000
25%        10.734351
50%        19.190072
75%        29.329962
max      1172.493976
Name: median_savings_for_Journey_route_in_Detected_Country, dtype: float64

# Mean Encoding

In [ ]:
# Calculate the mean encoding for Journey_route
mean_encoding = df_train.groupby('Journey_route')['median_savings_for_Journey_route_in_Detected_Country'].mean()

# Apply the mean encoding to both training and test datasets
df_train['Journey_route_mean_encoded'] = df_train['Journey_route'].map(mean_encoding)
df_test['Journey_route_mean_encoded'] = df_test['Journey_route'].map(mean_encoding)

In [ ]:
global_mean = df_train['median_savings_for_Journey_route_in_Detected_Country'].mean()
df_test['Journey_route_mean_encoded'].fillna(global_mean, inplace=True)


In [ ]:
# Calculate mean encoding for Detected_Country
country_mean_encoding = df_train.groupby('Detected_Country')['median_savings_for_Journey_route_in_Detected_Country'].mean()

# Calculate mean encoding for Detected_Currency
currency_mean_encoding = df_train.groupby('Detected_Currency')['median_savings_for_Journey_route_in_Detected_Country'].mean()

# Apply the mean encoding to both training and test datasets
df_train['Detected_Country_mean_encoded'] = df_train['Detected_Country'].map(country_mean_encoding)
df_test['Detected_Country_mean_encoded'] = df_test['Detected_Country'].map(country_mean_encoding)

df_train['Detected_Currency_mean_encoded'] = df_train['Detected_Currency'].map(currency_mean_encoding)
df_test['Detected_Currency_mean_encoded'] = df_test['Detected_Currency'].map(currency_mean_encoding)


In [ ]:
global_mean_country = df_train['median_savings_for_Journey_route_in_Detected_Country'].mean()
df_test['Detected_Country_mean_encoded'].fillna(global_mean_country, inplace=True)

global_mean_currency = df_train['median_savings_for_Journey_route_in_Detected_Country'].mean()
df_test['Detected_Currency_mean_encoded'].fillna(global_mean_currency, inplace=True)


In [ ]:
# Assuming you're including other features like 'days_until_departure', 'Detected_Country', and 'Detected_Currency'
# For simplicity, let's proceed with just 'Journey_route_mean_encoded' and 'days_until_departure' as examples
X_train = df_train[['Journey_route_mean_encoded', 'days_until_departure',]]
y_train = df_train['median_savings_for_Journey_route_in_Detected_Country']

X_test = df_test[['Journey_route_mean_encoded', 'days_until_departure']]
y_test = df_test['median_savings_for_Journey_route_in_Detected_Country']


In [ ]:
import xgboost as xgb

# Define your model
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 8, alpha = 10, n_estimators = 50)

# Train your model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")


Mean Absolute Error: 8.51535731182034
R-squared: 0.7538892419266212


# New

In [ ]:
import pandas as pd
df_1 = pd.read_csv('../data/4.processed_data/Query4_results_test_processed.csv')
df_2 = pd.read_csv('../data/4.processed_data/Query1503_results_conv_processed.csv')
df_3 = pd.read_csv('../data/4.processed_data/Query1803_results_processed.csv')
df_train = pd.concat([df_1,df_2,df_3])
df_test = pd.read_csv('../data/4.processed_data/Query1603_results_test_processed.csv')

In [ ]:
# Group by Flight_ID and calculate max, min prices and their absolute difference
price_stats_journey = df_train.groupby('Journey_route')['Price_in_USD'].agg(['max', 'min'])
price_stats_journey['max_abs_diff_Journey_route'] = price_stats_journey['max'] - price_stats_journey['min']
price_stats_journey.columns = ['max_price_Journey_route', 'min_price_Journey_route', 'max_abs_diff_Journey_route']
price_stats_journey['max_rel_diff_Journey_route'] = (price_stats_journey['max_abs_diff_Journey_route'] / price_stats_journey['min_price_Journey_route']) * 100

In [ ]:
df_train = pd.merge(df_train, price_stats_journey, on='Journey_route', how='left')

In [ ]:
df_train["abs_diff_to_min_price_Journey_route"] = df_train["Price_in_USD"] - df_train["min_price_Journey_route"]
df_train["rel_diff_to_min_price_Journey_route"] = ((df_train["Price_in_USD"] /df_train["min_price_Journey_route"] ) -1) * 100
df_train['rel_price_score_Journey_route'] = df_train['rel_diff_to_min_price_Journey_route'] / df_train['max_rel_diff_Journey_route']

In [ ]:
df_train.abs_diff_to_min_price_Journey_route.describe()

count    4729.000000
mean      497.427110
std       923.440485
min         0.000000
25%        74.260000
50%       186.244552
75%       470.957792
max      7381.591065
Name: abs_diff_to_min_price_Journey_route, dtype: float64

In [ ]:
df_train.rel_diff_to_min_price_Journey_route.describe()

count    4729.000000
mean       90.302816
std       139.548393
min         0.000000
25%        14.418003
50%        43.770431
75%       109.063318
max      1187.102731
Name: rel_diff_to_min_price_Journey_route, dtype: float64

In [ ]:
df_train.rel_diff_to_min_price_FlightID.describe()

count    4729.000000
mean        3.745923
std         9.592982
min         0.000000
25%         0.847017
50%         3.069277
75%         4.110414
max       257.689565
Name: rel_diff_to_min_price_FlightID, dtype: float64

In [ ]:
df_train.columns

Index(['airline_code', 'departure_airport_code', 'destination_airport_code',
       'ticket_price', 'departure_date', 'arrival_date', 'First_flight',
       'first_flight_code', 'last_flight_code', 'Detected_Language',
       'Detected_Country', 'Detected_Currency', 'Flight_ID', 'Price_in_USD',
       'commute_time', 'query_date', 'days_until_departure',
       'FlightID_in_Countries_Count', 'departure_date_day', 'arrival_date_day',
       'Journey_route', 'Journey_ID', 'max_price_FlightID',
       'min_price_FlightID', 'max_price_diff_FlightID',
       'max_rel_price_diff_FlightID', 'abs_diff_to_min_price_FlightID',
       'rel_diff_to_min_price_FlightID', 'rel_price_score_FlightID',
       'max_price_JourneyID', 'min_price_JourneyID', 'max_abs_diff_JourneyID',
       'max_rel_diff_Journey', 'abs_diff_to_min_price_JourneyID',
       'rel_diff_to_min_price_JourneyID', 'rel_price_score_JourneyID',
       'max_journey_same_country', 'min_journey_same_country',
       'max_abs_diff_perIDG

In [ ]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_route_train = df_train.groupby(['Journey_route'])['rel_diff_to_min_price_JourneyID'].median().reset_index(name='median_savings_for_Journey_route_in_Detected_Country')
# Merge the average_savings DataFrame back into df_reduced
df_train= df_train.merge(median_savings_route_train, on=['Journey_route'], how='left')

In [ ]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_route_test = df_test.groupby(['Journey_route'])['rel_diff_to_min_price_JourneyID'].median().reset_index(name='median_savings_for_Journey_route_in_Detected_Country')
# Merge the average_savings DataFrame back into df_reduced
df_test = df_test.merge(median_savings_route_test, on=['Journey_route'], how='left')

In [ ]:
# Calculate the mean encoding for Journey_route
mean_encoding = df_train.groupby('Journey_route')['median_savings_for_Journey_route_in_Detected_Country'].mean()

# Apply the mean encoding to both training and test datasets
df_train['Journey_route_mean_encoded'] = df_train['Journey_route'].map(mean_encoding)
df_test['Journey_route_mean_encoded'] = df_test['Journey_route'].map(mean_encoding)

In [ ]:
global_mean = df_train['median_savings_for_Journey_route_in_Detected_Country'].mean()
df_test['Journey_route_mean_encoded'].fillna(global_mean, inplace=True)


In [ ]:
# Calculate mean encoding for Detected_Country
country_mean_encoding = df_train.groupby('Detected_Country')['median_savings_for_Journey_route_in_Detected_Country'].mean()

# Calculate mean encoding for Detected_Currency
currency_mean_encoding = df_train.groupby('Detected_Currency')['median_savings_for_Journey_route_in_Detected_Country'].mean()

# Apply the mean encoding to both training and test datasets
df_train['Detected_Country_mean_encoded'] = df_train['Detected_Country'].map(country_mean_encoding)
df_test['Detected_Country_mean_encoded'] = df_test['Detected_Country'].map(country_mean_encoding)

df_train['Detected_Currency_mean_encoded'] = df_train['Detected_Currency'].map(currency_mean_encoding)
df_test['Detected_Currency_mean_encoded'] = df_test['Detected_Currency'].map(currency_mean_encoding)


In [ ]:
global_mean_country = df_train['median_savings_for_Journey_route_in_Detected_Country'].mean()
df_test['Detected_Country_mean_encoded'].fillna(global_mean_country, inplace=True)

global_mean_currency = df_train['median_savings_for_Journey_route_in_Detected_Country'].mean()
df_test['Detected_Currency_mean_encoded'].fillna(global_mean_currency, inplace=True)


In [ ]:
# Assuming you're including other features like 'days_until_departure', 'Detected_Country', and 'Detected_Currency'
# For simplicity, let's proceed with just 'Journey_route_mean_encoded' and 'days_until_departure' as examples
X_train = df_train[['Journey_route_mean_encoded', 'days_until_departure','Detected_Country_mean_encoded','Detected_Currency_mean_encoded']]
y_train = df_train['median_savings_for_Journey_route_in_Detected_Country']

X_test = df_test[['Journey_route_mean_encoded', 'days_until_departure','Detected_Country_mean_encoded','Detected_Currency_mean_encoded']]
y_test = df_test['median_savings_for_Journey_route_in_Detected_Country']


In [ ]:
import xgboost as xgb

# Define your model
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 8, alpha = 10, n_estimators = 50)

# Train your model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")


Mean Absolute Error: 17.357763709792035
R-squared: 0.2317343092038695


In [ ]:
df_train.median_savings_for_Journey_route_in_Detected_Country.describe()

count    4729.000000
mean       28.309959
std        37.921052
min         0.000000
25%        11.110648
50%        21.257776
75%        39.825343
max      1172.493976
Name: median_savings_for_Journey_route_in_Detected_Country, dtype: float64

In [ ]:
df_test.median_savings_for_Journey_route_in_Detected_Country.describe()

count    1781.000000
mean       30.588407
std        31.068700
min         0.000000
25%         9.947389
50%        16.938670
75%        30.829238
max       153.040929
Name: median_savings_for_Journey_route_in_Detected_Country, dtype: float64

In [ ]:
df_train.median_savings_for_Journey_route_in_Detected_Country.describe()

count    4729.000000
mean       26.319095
std        21.514394
min         0.649380
25%        12.243320
50%        21.368913
75%        39.949431
max       109.515476
Name: median_savings_for_Journey_route_in_Detected_Country, dtype: float64

In [ ]:
df_test.median_savings_for_Journey_route_in_Detected_Country.describe()

count    1781.000000
mean       30.316529
std        30.506234
min         3.995015
25%        13.745972
50%        21.508544
75%        30.040884
max       151.728050
Name: median_savings_for_Journey_route_in_Detected_Country, dtype: float64

In [73]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_route_train = df_train.groupby(['Journey_ID','Detected_Currency'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced


In [78]:
135*13

1755

In [83]:
median_savings_route_train.describe()

,median_savings_for_Journey_ID_country
count,1217.000000
mean,3.344232
std,9.235524
min,0.000000
25%,0.621318
50%,2.099948
75%,3.941009
max,185.298209


In [57]:
df_train.rel_diff_to_min_price_FlightID.describe()

count    5806.000000
mean        3.090059
std         8.683293
min         0.000000
25%         0.626945
50%         1.648590
75%         3.930153
max       257.689565
Name: rel_diff_to_min_price_FlightID, dtype: float64

In [53]:
df_train.Journey_route.nunique()

38

In [51]:
df_train.Journey_ID.nunique()

135

In [59]:
median_savings_route_train.describe()

,median_savings_for_Journey_ID
count,135.000000
mean,2.730387
std,3.316285
min,0.174945
25%,0.778689
50%,2.221997
75%,3.845189
max,26.013457


In [91]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_route_train = df_train.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced

In [92]:
df_train.rel_diff_to_min_price_FlightID

0        4.097040
1        4.111681
2        3.587679
3        4.429233
4        4.006443
          ...    
5801    27.232620
5802     0.603609
5803     0.000000
5804     0.099994
5805     0.113958
Name: rel_diff_to_min_price_FlightID, Length: 5806, dtype: float64

In [97]:
df_train.Journey_ID.describe()

count                               5806
unique                               135
top       FCO-MAD: 28-11-2024 28-11-2024
freq                                 140
Name: Journey_ID, dtype: object

In [98]:
df_train.Detected_Country.describe()

count        5806
unique         14
top       Schweiz
freq          640
Name: Detected_Country, dtype: object

In [99]:
14*135

1890

In [96]:
median_savings_route_train.describe()

,median_savings_for_Journey_ID_country
count,1219.000000
mean,3.340380
std,9.228428
min,0.000000
25%,0.622860
50%,2.095719
75%,3.940996
max,185.298209


In [94]:
df_train.columns

Index(['airline_code', 'departure_airport_code', 'destination_airport_code',
       'ticket_price', 'departure_date', 'arrival_date', 'First_flight',
       'first_flight_code', 'last_flight_code', 'Detected_Language',
       'Detected_Country', 'Detected_Currency', 'Flight_ID', 'Price_in_USD',
       'commute_time', 'query_date', 'days_until_departure',
       'FlightID_in_Countries_Count', 'departure_date_day', 'arrival_date_day',
       'Journey_route', 'Journey_ID', 'max_price_FlightID',
       'min_price_FlightID', 'max_price_diff_FlightID',
       'max_rel_price_diff_FlightID', 'abs_diff_to_min_price_FlightID',
       'rel_diff_to_min_price_FlightID', 'rel_price_score_FlightID',
       'max_price_JourneyID', 'min_price_JourneyID', 'max_abs_diff_JourneyID',
       'max_rel_diff_Journey', 'abs_diff_to_min_price_JourneyID',
       'rel_diff_to_min_price_JourneyID', 'rel_price_score_JourneyID',
       'max_journey_same_country', 'min_journey_same_country',
       'max_abs_diff_perIDG

In [125]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID = df_train.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')

# Merge the average_savings DataFrame back into df_reduced
mean_savings_Journey_ID.describe()

,median_savings_for_Journey_ID_country
count,1219.000000
mean,3.476366
std,8.807292
min,0.000000
25%,0.651815
50%,2.171812
75%,3.941188
max,169.333250


In [124]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route = df_train.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced
median_savings_Journey_route.describe()

,median_savings_for_Journey_ID_country
count,383.000000
mean,2.622178
std,3.153553
min,0.000000
25%,0.738425
50%,2.568839
75%,3.940677
max,47.872609


# Idea combining these two metrices

In [1]:
import pandas as pd
df_1 = pd.read_csv('../data/4.processed_data/Query4_results_test_processed.csv')
df_2 = pd.read_csv('../data/4.processed_data/Query1503_results_conv_processed.csv')
df_3 = pd.read_csv('../data/4.processed_data/Query1603_results_test_processed.csv')
df_train = pd.concat([df_1,df_2,df_3])
df_test = pd.read_csv('../data/4.processed_data/Query1803_results_processed.csv')

In [6]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID_train = df_train.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route_train = df_train.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_route_country')
# Merge the average_savings DataFrame back into df_reduced



# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID_test = df_test.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced

# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route_test = df_test.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_route_country')
# Merge the average_savings DataFrame back into df_reduced



In [10]:
# Step 1: Create a copy of df_reduced to work with
df_enriched = df_train.copy()

# Assuming mean_savings_Journey_ID and median_savings_Journey_route are as previously defined

# Step 2: Merge the mean_savings_for_Journey_ID_country DataFrame into df_enriched
df_enriched = df_enriched.merge(mean_savings_Journey_ID_train, on=['Journey_ID', 'Detected_Country'], how='left')

# Step 3: Merge the median_savings_Journey_route into df_enriched
# Note: Rename 'median_savings_for_Journey_ID_country' to 'median_savings_for_Journey_route' to avoid confusion
df_enriched = df_enriched.merge(median_savings_Journey_route_train, on=['Journey_route', 'Detected_Country'], how='left')

# Step 4: Calculate the simple mean of the two savings values
df_enriched['mean_savings'] = df_enriched[['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country']].mean(axis=1)


In [35]:

# Step 2: Merge the mean_savings_for_Journey_ID_country DataFrame into df_enriched
df_test = df_test.merge(mean_savings_Journey_ID_test, on=['Journey_ID', 'Detected_Country'], how='left')

# Step 3: Merge the median_savings_Journey_route into df_enriched
# Note: Rename 'median_savings_for_Journey_ID_country' to 'median_savings_for_Journey_route' to avoid confusion
df_test = df_test.merge(median_savings_Journey_route_test, on=['Journey_route', 'Detected_Country'], how='left')

# Step 4: Calculate the simple mean of the two savings values
df_test['mean_savings'] = df_test[['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country']].mean(axis=1)


KeyError: "None of [Index(['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country'], dtype='object')] are in the [columns]"

In [12]:
df_test.mean_savings.describe()

count    681.000000
mean       2.184869
std        3.128298
min        0.000000
25%        0.759336
50%        1.390440
75%        2.475880
max       22.675250
Name: mean_savings, dtype: float64

In [14]:
df_enriched.mean_savings.describe()

count    5806.000000
mean        2.630391
std         4.413457
min         0.000000
25%         0.751915
50%         1.981605
75%         3.716443
max       107.490743
Name: mean_savings, dtype: float64

In [26]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train = pd.get_dummies(df_enriched[['Journey_route','days_until_departure', 'Detected_Country']])
y_train = df_enriched['mean_savings']
feature_names = X_train.columns.tolist()



# Specify your model parameters here
params_grid = {'colsample_bytree': 0.9, 'eta': 0.299, 'max_depth': 7, 'subsample': 0.8}
params_optuna = {'max_depth': 7, 'eta': 0.319923163390001, 'subsample': 0.8, 'colsample_bytree': 0.9}
params_optuna200 = {'max_depth': 8, 'eta': 0.34991201767803204, 'subsample': 0.9, 'colsample_bytree': 0.9}


params_self = {'max_depth': 3, 'eta': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.9}




# Create XGBoost model
model_v8_combined  = xgb.XGBRegressor(**params_optuna200)

# Fit the model on your training data
model_v8_combined.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.34991201767803204,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [27]:
# Drop rows where 'mean_savings' is NaN
df_test= df_test.dropna(subset=['mean_savings'])


In [28]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Assuming new_test_df is your new test data


# Update X_test and y_test accordingly
X_test = pd.get_dummies(df_test[['Journey_route', 'days_until_departure', 'Detected_Country']])
y_test = df_test['mean_savings']


# Align the new test set features with the training features
# Add missing columns in the new test set
missing_cols = set(feature_names) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0

# Remove any columns in the new test set that are not present in the training set
X_test = X_test[feature_names]

# Now, you can predict with your model
y_pred = model_v8_combined.predict(X_test)



# Calculate and print the Mean Squared Error
mea = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mea}")


r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 0.4913521313429161
R-squared: 0.8402507674697541


just median

In [32]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train = pd.get_dummies(df_enriched[['Journey_route','days_until_departure', 'Detected_Country']])
y_train = df_enriched['mean_savings_for_Journey_ID_country']
feature_names = X_train.columns.tolist()



# Specify your model parameters here
params_grid = {'colsample_bytree': 0.9, 'eta': 0.299, 'max_depth': 7, 'subsample': 0.8}
params_optuna = {'max_depth': 7, 'eta': 0.319923163390001, 'subsample': 0.8, 'colsample_bytree': 0.9}
params_optuna200 = {'max_depth': 8, 'eta': 0.34991201767803204, 'subsample': 0.9, 'colsample_bytree': 0.9}


params_self = {'max_depth': 3, 'eta': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.9}




# Create XGBoost model
model_v8_combined  = xgb.XGBRegressor(**params_optuna200)

# Fit the model on your training data
model_v8_combined.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.34991201767803204,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [34]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Assuming new_test_df is your new test data


# Drop rows where 'mean_savings' is NaN
# Drop rows where 'mean_savings' is NaN
df_test= df_test.dropna(subset=['mean_savings_for_Journey_ID_country'])

#df_test= df_test.dropna(subset=['mean_savings'])
X_test = pd.get_dummies(df_test[['Journey_route', 'days_until_departure', 'Detected_Country']])
y_test = df_test['mean_savings_for_Journey_ID_country']


# Align the new test set features with the training features
# Add missing columns in the new test set
missing_cols = set(feature_names) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0

# Remove any columns in the new test set that are not present in the training set
X_test = X_test[feature_names]

# Now, you can predict with your model
y_pred = model_v6_combined.predict(X_test)



# Calculate and print the Mean Squared Error
mea = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mea}")


r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 1.3204493832928457
R-squared: 0.7429883678819165
